#### 불균형 데이터 셋 문제(imbalanced data problem)
다수 클래스를 택하는 모형의 정확도가 높아져 예측 정확도가 떨어질 수 있음   
소수 클래스의 경우 정확도는 높지만 재현율(recall)이 작아질 수 있음    
(100개의 샘플이 0 80개, 1 20개로 구성되어 있을 때 0은 다 맞추고 1은 10개를 틀렸을 경우   
=>정확도는 90%이지만 20개의 1중에서 10개만 맞췄기때문에 recall은 0.5가 되므로 좋은 모형이라고 할 수 없음)   
* precision(정확도) = tp/tp+fp(예측=실제 1/예측1)
* recall(재현율) = tp/tp+fn(예측=실제 1/실제1)   
불균형 데이터는 언더 샘플링, 오버 샘플링, 복합 샘플링 등의 방법으로 불균형을 해소시킨 후 학습을 시키면 예측 정확도가 향상될 수 있음
---

1. 불균형 데이터셋

In [1]:
import pandas as pd
from sklearn.datasets import make_classification
X,y = make_classification(n_samples=10000, n_features=2,n_redundant=0,n_clusters_per_class=1,weights=[0.99],flip_y=0,random_state=1)
dfX=pd.DataFrame(X,columns=['a','b'])
dfy=pd.DataFrame(y,columns=['y'])
df=pd.concat([dfX,dfy],axis=1)
df

,a,b,y
0,0.222014,0.540207,0
1,1.347439,1.412824,0
2,0.537238,0.372730,0
3,2.134462,1.404819,0
4,2.315827,1.356858,0
...,...,...,...
9995,2.440385,1.695643,0
9996,-0.790502,0.194243,0
9997,1.878130,0.829500,0
9998,2.585933,1.927995,0


In [2]:
X1 = df[['a','b']] #독립변수
y1 = df['y']

In [ ]:
df['y'].value_counts() #불균형 데이터셋

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, stratify=y1, random_state=10)

#불균형 데이터셋으로 만든 모형
model1 = LogisticRegression(random_state=42)
model1.fit(X_train,y_train)

LogisticRegression(random_state=42)

In [4]:
print('학습용:',model1.score(X_train,y_train))
print('검증용:',model1.score(X_test,y_test))

학습용: 0.994125
검증용: 0.995


In [6]:
pred1 = model1.predict(X_test)
pred1

array([0, 0, 0, ..., 0, 0, 0])

In [7]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred1)
cm
#score는 0.9945로 높으나 recall의 경우 10/(10+10) = 0.5로 낮아지는 문제가 발생함

array([[1980,    0],
       [  10,   10]])

In [8]:
from sklearn.metrics import classification_report, confusion_matrix
#소수 클래스의 정확도와 Precision , recall, f1-score확인
print(classification_report(y_test,pred1))
#모형의 전반적인 정확도(accuracy)는 높지만 소수 클래스의 재현율(recall)이 0.5로 낮은 문제점

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1980
           1       1.00      0.50      0.67        20

    accuracy                           0.99      2000
   macro avg       1.00      0.75      0.83      2000
weighted avg       1.00      0.99      0.99      2000



In [9]:
#균형 데이터
X,y = make_classification(n_samples=10000, n_features=2,n_redundant=0,n_clusters_per_class=1,flip_y=0,random_state=1)
dfX = pd.DataFrame(X,columns=['a','b'])
dfy = pd.DataFrame(y,columns=['y'])
df2 = pd.concat([dfX,dfy],axis=1)

In [10]:
df2['y'].value_counts()

0    5000
1    5000
Name: y, dtype: int64

In [11]:
X2 = df2[['a','b']] #독립변수
y2 = df2['y']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X2,y2,test_size=0.2, stratify=y2,random_state=10)

model2 = LogisticRegression(random_state=42)
model2.fit(X_train,y_train)

LogisticRegression(random_state=42)

In [14]:
print('학습용:',model2.score(X_train,y_train))
print('검증용:',model2.score(X_test,y_test))

학습용: 0.896125
검증용: 0.891


In [15]:
pred2 = model2.predict(X_test)
pred2

array([1, 1, 0, ..., 0, 1, 0])

In [16]:
print(classification_report(y_test,pred2))
#정확도와 재현율이 비슷하게 처리됨

              precision    recall  f1-score   support

           0       0.88      0.91      0.89      1000
           1       0.90      0.87      0.89      1000

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000



In [17]:
len(X1),len(y1),len(pred1)

(10000, 10000, 2000)

In [ ]:
#비대칭 데이터는 언더샘플링, 오버샘플링, 복합샘플링 등의 방법으로 데이터 비율을 맞추면 정밀도가 향상된다.

#### 언더샘플링 vs. 오버샘플링
![sampling](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F2yuj6%2FbtqGirRpGhD%2FhmIPmKkQ6WMJ0PbpZZ5Wk0%2Fimg.png)  
![sampling2](https://www.datasciencecentral.com/wp-content/uploads/2021/10/2808331754.png)    
- 데이터가 불균형한 분포를 가지는 경우, 모델의 학습이 제대로 이루어지지 않을 수 있다. 이를 해결하기 위해 Undersampling, Oversampling기법을 이용
- Undersamlping : 다수 범주의 데이터를 소수 범주의 데이터 수에 맞게 줄이는 샘플링 방식 
    - Random Sampling : 다수 범주에서 무작위로 샘플링
    - Tomek Links : 두 범주 사이를 탐지하고 정리, 부정확한 분류경계선을 방지
    - CNN Rule : 소수 범주 전체와 다수 범주 중 임의로 선택한 하나의 데이터를 이용, 서비 데이터를 생성
    - Ons Sided Selection : Tomek Links + CNN Rule 
    - 장점 : 다수 범주 데이터의 제거로 계산시간이 감소
    - 단점 : 데이터 제거로 인한 정보 손실 발생 가능

- Oversampling : 소수 범주의 데이터를 다수 범주의 데이터 수에 맞게 늘리는 샘플링 방식
    - Resampling : 소수 범주의 데이터 수를 다수 범주의 데이터 수와 비슷해지도록 증가시키는 방식
    - SMOTE : 소수 범주에서 가상의 데이터를 생성하는 방법
    - Borderline SMOTE : Borderline 부분에 대해서만 SMOTE방식을 사용
    - ADASYN : Borderline SMOTE과 비슷하지만 샘플링 개수를 데이터 위치에 따라 다르게 설정하는 차이가 있음
    - GAN : 생성자와 구분자로 구성, 모델은 딥러닝을 사용하는 최신 오버 샘플링 기법
    - 장점 : 데이터를 증가시키기 때문에 정보 손실이 없음. 대부분의 경우 언더 샘플링에 비해 높은 분류 정확도
    - 단점 : 데이터 증가로 인해 계산 시간이 증가할 수 있으며 과적합 가능성이 존재. 노이즈 또는 이상치에 민감
- 출처
    - https://hwi-doc.tistory.com/entry/%EC%96%B8%EB%8D%94-%EC%83%98%ED%94%8C%EB%A7%81Undersampling%EA%B3%BC-%EC%98%A4%EB%B2%84-%EC%83%98%ED%94%8C%EB%A7%81Oversampling
    - https://casa-de-feel.tistory.com/15
    - https://www.datasciencecentral.com/handling-imbalanced-data-sets-in-supervised-learning-using-family/

In [18]:
X,y = make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)

---
2. 언더샘플링 : 데이터의 손실이 크고 중요한 특성을 가진 데이터를 잃을 수 있음

RandomUnderSampler   
![randomundersampler](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbrBgjg%2FbtrazujrZg6%2FRbK9sY5Ei4s7YaaqvFPUB0%2Fimg.png)

In [23]:
#무작위로 다수 클래스의 데이터를 없애는 단순 샘플링
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
X_sample, y_sample = RandomUnderSampler(random_state=0).fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample,columns=[['a','b']])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    100
1    100
Name: y, dtype: int64

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_samp,y_samp,test_size=0.2,stratify=y_samp,random_state=10)
model3 = SVC(random_state=42)
model3.fit(X_train,y_train)

/opt/homebrew/anaconda3/envs/virt/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/opt/homebrew/anaconda3/envs/virt/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(random_state=42)

In [25]:
print('학습용:',model3.score(X_train,y_train))
print('검증용:',model3.score(X_test,y_test))

학습용: 0.89375
검증용: 0.925


/opt/homebrew/anaconda3/envs/virt/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/opt/homebrew/anaconda3/envs/virt/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [26]:
pred3 = model3.predict(X_test)

/opt/homebrew/anaconda3/envs/virt/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [27]:
print(classification_report(y_test,pred3))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92        20
           1       0.90      0.95      0.93        20

    accuracy                           0.93        40
   macro avg       0.93      0.93      0.92        40
weighted avg       0.93      0.93      0.92        40



Tomek's Link   
![tomeklink1](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbqALfi%2Fbtrat29dNj5%2FKzQBX9IVc5leUsq249Wkh1%2Fimg.png)   
![tomeklink2](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FvrFpB%2Fbtrav7CriOV%2F40dG0kK9unkJCHuu01RwKK%2Fimg.jpg)
![tomeklink3](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbxHJZU%2Fbtrazuw5iqm%2FZOLbCTrhuq5ItwKxeNt3n1%2Fimg.jpg)

In [ ]:
#토멕링크(Tomek's link) : 서로 다른 클래스에 속하는 한 쌍의 데이터
#토멕링크 중에서 다수 클래스에 속한 샘플을 제거함으로써 데이터의 균형을 맞추는 방법
#'majority' : 다수 클래스의 샘플을 제거
#'not minority' : 소수 클래스를 제외하고 샘플링
#'not majority' : 다수 클래스를 제외하고 샘플링
#'all' : 모든 클래스를 샘플링
#'auto' : not minority와 같음(기본 옵션) 

In [28]:
from imblearn.under_sampling import TomekLinks

X_sample,y_sample = TomekLinks(sampling_strategy='majority').fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample,columns=['a','b'])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()
#토멕링크 중에서 다수 클래스의 샘플들을 제거하는 방식, 1:1로 맞추는 방식은 아님

0    9874
1     100
Name: y, dtype: int64

CNN Rule   
![cnn1](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FwdUVi%2FbtrazLyS0go%2FBiKdMbi2kiFa6kuAbckwak%2Fimg.jpg)

In [ ]:
#CNN(Condensed Nearest Neighbour) : 1-NN모형으로 분류되지 않는 데이터만 남기는 방법
#다수의 데이터 중에서 하나를 골라서 최근접 이웃이 다수 클래스이면 그 샘플을 빼는 방식
#시간이 많이 걸림

In [29]:
from imblearn.under_sampling import CondensedNearestNeighbour

X_sample, y_sample = CondensedNearestNeighbour(random_state=0).fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample, columns=['a','b'])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()
#1:1로 맞춰지지는 않음

0    187
1    100
Name: y, dtype: int64

One Sided Selection   
![oss](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FJ7rq6%2FbtraBegvVZR%2FeaxdKs5ikiNJOKRSnSBQEK%2Fimg.jpg)

In [ ]:
#One Sided Selection 
#토멕링크 방법과 CNN방법을 섞은 방식
#토멕링크 중 다수 클래스의 샘플을 제거하고 나머지 데이터 중에서도 서로 붙어있는 다수 클래스 데이터는 1-NN방법으로 제외하는 방식

In [30]:
from imblearn.under_sampling import OneSidedSelection
X_sample,y_sample = OneSidedSelection(random_state=0).fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample, columns=['a','b'])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    6593
1     100
Name: y, dtype: int64

ENN   
![enn](https://slideplayer.com/slide/13128031/79/images/14/Editing+Nearest+Neighbor+%28ENN%29.jpg)   
출처 : https://slideplayer.com/slide/13128031/

In [ ]:
#ENN(Edited Nearest Neighbours)
#다수 클래스 데이터 중 소수 클래스와 가장 가까운 k(n_neighbors)개의 데이터가 모두 또는 다수 클래스가 아니면 삭제하는 방법
#소수 클래스 주변의 다수 클래스 데이터는 삭제됨

In [31]:
from imblearn.under_sampling import EditedNearestNeighbours
#kind_sel='all' 모두
#kind_sel='mode'다수
#5개의 이웃이 모두 같은 클래스가 아니면 그 샘플을 제거함
X_sample,y_sample = EditedNearestNeighbours(kind_sel='all',n_neighbors=5).fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample, columns=['a','b'])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    9747
1     100
Name: y, dtype: int64

NCR   
![ncr1](https://player.slideplayer.com/79/13128031/slides/slide_16.jpg)
![ncr2](https://player.slideplayer.com/79/13128031/slides/slide_18.jpg)

In [ ]:
#Neighbourhood Cleaning Rule
#CNN과 ENN방법을 섞은 것

In [32]:
from imblearn.under_sampling import NeighbourhoodCleaningRule

#kind_sel='all' 모두
#kind_sel='mode'다수
X_sample,y_sample = NeighbourhoodCleaningRule(kind_sel='all',n_neighbors=5).fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample, columns=['a','b'])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    9721
1     100
Name: y, dtype: int64

---
3. 오버샘플링

In [33]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)

RandomOverSampler  
![resampling](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbLFPVl%2FbtraBUa2mJ3%2FcAMqr8afEfnV6cHePuixAk%2Fimg.jpg)

In [34]:
#무작위로 소수 클래스의 데이터를 복제하여 늘리는 방식
#정보가 손실되지 않으나 과적합이 될 수 있음
from imblearn.over_sampling import RandomOverSampler
import pandas as pd 

X_sample,y_sample = RandomOverSampler(random_state=0).fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample, columns=['a','b'])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    9900
1    9900
Name: y, dtype: int64

ADASYN   
![asasyn](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FpgDyc%2FbtrazuRlSo8%2FWuex9ZG0jUllFCxR8TO2k0%2Fimg.png)

In [ ]:
#ADASYN(Adaptive Synthetic Sampling)
#소수 클래스 데이터와 그 데이터에서 가장 가까운 k개의 소수 클래스 데이터 중 
#무작위로 선택된 데이터 사이의 직선상에서 가장의 소수 클래스 데이터를 만드는 방법

In [35]:
from imblearn.over_sampling import ADASYN
import pandas as pd

X_sample,y_sample = ADASYN(random_state=0).fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample, columns=['a','b'])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    9900
1    9899
Name: y, dtype: int64

SMOTE(가장 많이 사용되고 있는 방식)   
![smote](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FSsZa2%2FbtrawAYCbu2%2FEIDSWOVKfMzDrxdDn6A5n1%2Fimg.jpg)

In [36]:
#SMOTE(Synthetic Minority Over-sampling Technique) : 가장 많이 사용되고 있는 방식
#소수 클래스 샘플의 k 최근접 이웃을 찾는다.
#현재 샘플과 k개 이웃 간의 거리를 구하고 거리에 0~1사이의 임의의 값을 곱하여 소수 클래스의 샘플에 추가
#결과적으로 소수 클래스의 샘플을 주변의 이웃을 고려해 약간씩 이동시킨 포인트들을 추가하는 방식
from imblearn.over_sampling import SMOTE
import pandas as pd

X_sample,y_sample = SMOTE(random_state=0).fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample, columns=['a','b'])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()


0    9900
1    9900
Name: y, dtype: int64

---
4. 복합샘플링

SMOTE + ENN     
![smoteenn](https://media.springernature.com/original/springer-static/image/chp%3A10.1007%2F978-981-15-6048-4_4/MediaObjects/491575_1_En_4_Fig3_HTML.png)   
출처 : https://link.springer.com/chapter/10.1007/978-981-15-6048-4_4

In [ ]:
#SMOTE+ENN 
#SMOTE : 소수 클래스의 샘플을 주변의 이웃을 고려해 약간씩 이동시킨 포인트들을 추가
#ENN : 소수 클래스의 주변의 다수 클래스 데이터를 삭제

In [37]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)

In [38]:
from imblearn.combine import SMOTEENN
import pandas as pd

X_sample,y_sample = SMOTEENN(random_state=0).fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample, columns=['a','b'])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

1    8941
0    8645
Name: y, dtype: int64

SOMTE + TOMEK LINKS   
![smotetomek](https://slideplayer.com/slide/13128031/79/images/33/Smote+%2B+Tomek+links.jpg)

In [ ]:
#SMOTE+Tomek
#SMOTE : 소수 클래스의 샘플을 주변의 이웃을 고려해 약간씩 이동시킨 포인트들을 추가
#토멕링크 : 토멕링크 중에서 다수 클래스의 샘플들을 제거

In [39]:
from imblearn.combine import SMOTETomek
import pandas as pd

X_sample,y_sample = SMOTETomek(random_state=0).fit_resample(X,y)

X_samp = pd.DataFrame(data=X_sample, columns=['a','b'])
y_samp = pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    9653
1    9653
Name: y, dtype: int64